In [0]:
from pyspark.sql import DataFrame
from pyspark.sql.functions import *
from typing import List, Dict
import uuid
from pyspark.sql.functions import *
from pyspark.sql.types import *
import json
import os
import datetime
import uuid
from functools import reduce

class DataQualityValidator:
    def __init__(self, df: DataFrame, notebook_name: str):
        self.df = df
        self.notebook_name = notebook_name
        self.timestamp = spark.sql("SELECT current_timestamp() AS ts").collect()[0]["ts"]
        self.results = []

    def check_not_null_or_blank(self, rule: Dict):
        failed_df = self.df.filter(
            col(rule["column"]).isNull() | (trim(col(rule["column"])) == "")
        ).withColumn("error", lit(rule["description"])) \
         .withColumn("rule_id", lit(rule["id"])) \
         .withColumn("category", lit(rule["category"])) \
         .withColumn("severity", lit(rule["severity"])) \
         .withColumn("notebook_name", lit(self.notebook_name)) \
         .withColumn("timestamp", lit(str(self.timestamp)))

        self.results.append({
            "rule_id": rule["id"],
            "rule": rule["description"],
            "category": rule["category"],
            "severity": rule["severity"],
            "failed_count": failed_df.count(),
            "failed_records": failed_df
        })

    def check_pattern(self, rule: Dict):
        failed_df = self.df.filter(~col(rule["column"]).rlike(rule["pattern"])) \
            .withColumn("error", lit(rule["description"])) \
            .withColumn("rule_id", lit(rule["id"])) \
            .withColumn("category", lit(rule["category"])) \
            .withColumn("severity", lit(rule["severity"])) \
            .withColumn("notebook_name", lit(self.notebook_name)) \
            .withColumn("timestamp", lit(str(self.timestamp)))

        self.results.append({
            "rule_id": rule["id"],
            "rule": rule["description"],
            "category": rule["category"],
            "severity": rule["severity"],
            "failed_count": failed_df.count(),
            "failed_records": failed_df
        })

    def check_custom_sql(self, rule: Dict):
        expression = rule.get("expression")
        failed_df = self.df.filter(expr(expression)) \
            .withColumn("error", lit(rule.get("description", "Custom SQL check failed"))) \
            .withColumn("rule_id", lit(rule.get("id", "NA"))) \
            .withColumn("category", lit(rule.get("category", "unspecified"))) \
            .withColumn("severity", lit(rule.get("severity", "warning"))) \
            .withColumn("notebook_name", lit(self.notebook_name)) \
            .withColumn("timestamp", lit(str(self.timestamp)))

        self.results.append({
            "rule_id": rule.get("id", "NA"),
            "rule": rule.get("description", "Custom SQL check failed"),
            "category": rule.get("category", "unspecified"),
            "severity": rule.get("severity", "warning"),
            "failed_count": failed_df.count(),
            "failed_records": failed_df
    })

    def run_checks(self, rules: List[Dict[str, any]]) -> DataFrame:
        failed_dfs = []

        # Execute each rule type
        for rule in rules:
            if rule["type"] == "not_null_or_blank":
                self.check_not_null_or_blank(rule)
            elif rule["type"] == "pattern":
                self.check_pattern(rule)
            elif rule["type"] == "custom_sql":
                self.check_custom_sql(rule)

        # Collect failed records from results
        for res in self.results:
            failed_dfs.append(res["failed_records"].select(self.df.columns))

        if failed_dfs:
            # Drop extra columns from each failed_df
            trimmed_failed_dfs = [
                df.drop("error", "rule_id", "category", "severity", "notebook_name", "timestamp")
                for df in failed_dfs
            ]

            # Union all failed records safely
            if len(trimmed_failed_dfs) == 1:
                all_failed_df = trimmed_failed_dfs[0].dropDuplicates()
            else:
                all_failed_df = reduce(
                    lambda df1, df2: df1.unionByName(df2),
                    trimmed_failed_dfs
                ).dropDuplicates()

            # Separate passed and failed records
            passed_df = self.df.subtract(all_failed_df).withColumn("DQ_Status", lit("Y"))
            failed_df = all_failed_df.withColumn("DQ_Status", lit("N"))

            # Combine final DataFrame
            self.df = passed_df.unionByName(failed_df)
        else:
            # All records passed
            self.df = self.df.withColumn("DQ_Status", lit("Y"))

        return self.df
    
    def show_results(self):
        for res in self.results:
            print(f"[{res['severity'].upper()}] Rule: {res['rule']} | Failed Count: {res['failed_count']}")
            res['failed_records'].show(truncate=False)

    def get_summary_df(self) -> DataFrame:
        summary_data = [(r["rule_id"], r["rule"], r["category"], r["severity"],
                         r["failed_count"], self.notebook_name, str(self.timestamp)) for r in self.results]

        return spark.createDataFrame(summary_data, schema=[
            "rule_id", "rule", "category", "severity", "failed_count", "notebook_name", "timestamp"
        ])

    def log_errors_to_blob(self, base_path: str):
        # Format timestamp for folder naming
        now = datetime.datetime.now()
        timestamp_str = now.strftime("%Y-%m-%dT%H-%M-%S")
        reject_path = f"{base_path}/rejects_{timestamp_str}"

        #write the summary into reject
        summary_df = self.get_summary_df()
        summary_df.write.mode("overwrite").option("header", "true").csv(f"{reject_path}/summary")

        #append into validation_summary table
        batch_id = str(uuid.uuid4())
        summary_df = summary_df.withColumn("batch_id", lit(batch_id))
        summary_df.write.format("delta").mode("append").saveAsTable("workspace.myschema.validation_summary")
        #write the data into reject
        for res in self.results:
            if res["failed_records"].count() > 0:
                res["failed_records"].write.mode("append").json(reject_path)
